In [4]:
#classification

#Train test Spit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#Logistic Regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', multi_class='auto')
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

#K-Nearest Neighbor
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors=10)

#Support Vector Classifier
from sklearn.svm import SVC
model_svc = SVC(gamma='scale')

#Decission Tree Classifier
from sklearn.tree import DecisionTreeClassifier
model_dt = DecisionTreeClassifier()

#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=50)

#Naive Bayes
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()

#XGBoost
import xgboost as xg
model_xgb = xg.XGBClassifier(random_state=1)



#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10)

#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#Label Encoding
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
for i in Categorical:
    df[i] = LE.fit_transform(df[i])
    
#One hot encode
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

# logarithmic all data numeric
for i in numerical:
    df[i] = np.log(df[i]+1)
    

#PLOT Correlation
corr = data.corr()
plt.figure(figsize=(20,15))
sns.heatmap(corr, annot=True, annot_kws={'size':9})
plt.title('Correlation Clean Data')
plt.show()


#Feature Selection
for col in feat_corr['mytsel_lapse'].index[1:]:
    # Pick desired cutoff for dropping features. In absolute-value terms.
    if np.absolute(feat_corr.loc['mytsel_lapse',col]) < 0.1:
        # Drop the feature if correlation is below cutoff
        data.drop(columns=col, inplace=True)
print('-'*40)
print('\nData shape after feature selection:', data.shape)
print('\nCounts of lapse VS non-lapse in new data:')
print(data['mytsel_lapse'].value_counts())
print('-'*40)

#SMOTE
from imblearn.combine import SMOTETomek
sm = SMOTETomek(random_state=42, sampling_strategy='minority')
X_train_upsamp, Y_train_upsamp = sm.fit_sample(X_train, y_train)


#Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))

#GridSearch
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
              {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

In [ ]:
#optuna
xgb = XGBClassifier(
    n_estimators=278,
    max_depth=7,
    min_child_weight=1,
    colsample_bytree=0.20651439788177967,
    scale_pos_weight=2.35,
    reg_alpha=3.6754725664416057,
    reg_lambda=6.401966398857545,
    gamma=0.2160029174594188,
    learning_rate=0.1885401400753244
)

In [ ]:
#Handling Outlier
# Removing Outliers from high-correlation features
new_corr = data_reeng.corr()
cols = new_corr['mytsel_lapse'].index[1:]

# For each feature correlated with Class...
for col in cols:
    # If absolute correlation value is more than X percent...
    correlation = new_corr.loc['mytsel_lapse',col]
    if np.absolute(correlation) > 0.1:
        
        # Separate the classes of the high-correlation column
        nonlapse = no_outliers.loc[no_outliers['mytsel_lapse']==0,col]
        lapse = no_outliers.loc[no_outliers['mytsel_lapse']==1,col]

        # Identify the 25th and 75th quartiles
        all_values = no_outliers.loc[:,col]
        q25, q75 = np.percentile(all_values, 25), np.percentile(all_values, 75)
        # Get the inter quartile range
        iqr = q75 - q25
        # Smaller cutoffs will remove more outliers
        cutoff = iqr * 1.5
        # Set the bounds of the desired portion to keep
        lower, upper = q25 - cutoff, q75 + cutoff
        
        # If positively correlated...
        # Drop nonlapse above upper bound, and lapse below lower bound
        if correlation > 0: 
            no_outliers.drop(index=nonlapse[nonlapse>upper].index,inplace=True)
            no_outliers.drop(index=lapse[lapse<lower].index,inplace=True)
        
        # If negatively correlated...
        # Drop nonlapse below lower bound, and lapse above upper bound
        elif correlation < 0: 
            no_outliers.drop(index=nonlapse[nonlapse<lower].index,inplace=True)
            no_outliers.drop(index=lapse[lapse>upper].index,inplace=True)
        
print('\nData shape before removing outliers:', data_reeng.shape)
print('\nCounts of lapse VS non-lapse in previous data:')
print(data_reeng['mytsel_lapse'].value_counts())
print('-'*40)
print('-'*40)
print('\nData shape after removing outliers:', no_outliers.shape)
print('\nCounts of lapse VS non-lapse in new data:')
print(no_outliers['mytsel_lapse'].value_counts())